# Управление роботом PUMA

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

Возьмем готовые функции для моделирования кинематики:

In [ ]:
irb_l = [352.0, 70.0, 350.0, 380.0, 65.0]

In [ ]:
def irb_chain(q, l):
    base = Transform.identity()
    column = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1))
    )
    shoulder = column + Transform(
        Vector(l[1], 0, 0),
        Quaternion.from_angle_axis(q[1], Vector(0, -1, 0))
    )
    elbow = shoulder + Transform(
        Vector(0, 0, l[2]),
        Quaternion.from_angle_axis(q[2], Vector(0, 1, 0))
    )
    wrist = elbow + Transform(
        Vector(l[3], 0, 0),
        Quaternion.from_angle_axis(q[3], Vector(1, 0, 0)) *
        Quaternion.from_angle_axis(q[4], Vector(0, 1, 0))
    )
    flange = wrist + Transform(
        Vector(l[4], 0, 0),
        Quaternion.from_angle_axis(q[5], Vector(1, 0, 0)) *
        Quaternion.from_angle_axis(np.pi / 2, Vector(0, 1, 0))
    )
    return [
        base,
        column,
        shoulder,
        elbow,
        wrist,
        flange
    ]

In [ ]:
def wrap_from_to(value, s, e):
    r = e - s
    return value - (r * np.floor((value - s) / r))

def irb_ik(target, l, i=[1, 1, 1]):
    wrist = target + Vector(0, 0, -l[4]) +  Vector(0, 0, -l[0])
    projection = Vector(wrist.x, wrist.y, 0)
    q0 = Vector(0, 1, 0).angle_to(projection, Vector(0, 0, 1)) - np.pi / 2 * i[0] + np.pi
    d = ((projection.magnitude() - i[0] * l[1]) ** 2 + wrist.z ** 2) ** 0.5
    q2 = -i[1] * np.arccos(
        (l[2] **  2 + l[3] ** 2 - d ** 2) /\
        (2 * l[2] * l[3])
    ) + np.pi / 2
    triangle_angle = np.arcsin(
        l[3] * i[0] * np.sin(q2 - np.pi / 2) / d
    )
    lift_angle = np.arctan2(
        wrist.z,
        (projection.magnitude() - i[0] * l[1])
    )
    q1 = -i[0] * (np.pi / 2 + triangle_angle - lift_angle)
    ori = Quaternion.from_angle_axis(q0, Vector(0, 0, 1)) *\
        Quaternion.from_angle_axis(q1, Vector(0, -1, 0)) *\
        Quaternion.from_angle_axis(q2, Vector(0, 1, 0))
    ez = ori * Vector(1, 0, 0)
    ey = ori * Vector(0, 1, 0)
    tz = target.rotation * Vector(0, 0, 1)
    ty = target.rotation * Vector(0, 1, 0)
    wy = ez.cross(tz)
    q3 = ey.angle_to(wy, ez) + np.pi / 2 - np.pi / 2 * i[2]
    q4 = ez.angle_to(tz, wy) * i[2]
    q5 = wy.angle_to(ty, tz) + np.pi / 2 -np.pi / 2 * i[2]
    return (
        wrap_from_to(q0, -np.pi, np.pi),
        wrap_from_to(q1, -np.pi, np.pi),
        wrap_from_to(q2, -np.pi, np.pi),
        wrap_from_to(q3, -np.pi, np.pi),
        wrap_from_to(q4, -np.pi, np.pi),
        wrap_from_to(q5, -np.pi, np.pi)
    )

Добавим функцию учитывающую ограничения степеней подвижности:

In [ ]:
irb_lim = [
    (-180, 180),
    (-90, 110),
    (-230, 50),
    (-200, 200),
    (-115, 115),
    (-400, 400)
]

In [ ]:
def irb_ik_lim(target, l, i=[1, 1, 1]):
    solution = irb_ik(target, l, i)
    for index in range(len(solution)):
        if solution[index] < np.deg2rad(irb_lim[index][0]) or\
            solution[index] > np.deg2rad(irb_lim[index][1]) or\
            np.isnan(solution[index]):
            return None
    return solution

Объявим функцию для описания линейной траектории:

In [ ]:
def lin(start, end, t, total):
    return Transform.lerp(
        start,
        end,
        t / total
    )

In [ ]:
s = Transform(
    Vector(200, 400, 600),
    Quaternion.from_angle_axis(np.pi / 2, Vector(-1, 0, 0))
)
e = Transform(
    Vector(200, -300, 800),
    Quaternion.from_angle_axis(np.pi / 2, Vector(0, 1, 0))
)
irb_i = [1, 1, -1]

Рассмотрим это линейное движение:

In [ ]:
(x, y, z) = graphics.chain_to_points(
    irb_chain([0, 0, 0, 0, 0, 0], irb_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s, 100)
graphics.axis(ax, e, 100)

total = 100

def animate(frame):
    trs = lin(s, e, frame, total)
    q = irb_ik_lim(
        trs,
        irb_l,
        irb_i
    )
    if q != None:
        chain = irb_chain(q, irb_l)
        (x, y, z) = graphics.chain_to_points(chain)
        lines.set_data_3d(x, y, z)


animate(0)
fps = 25
irb_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(irb_ani.to_jshtml())

Повторим это движение в режиме переброски:

In [ ]:
(x, y, z) = graphics.chain_to_points(
    irb_chain([0, 0, 0, 0, 0, 0], irb_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s, 100)
graphics.axis(ax, e, 100)

total = 100

s_q = irb_ik_lim(s, irb_l, irb_i)
e_q = irb_ik_lim(e, irb_l, irb_i)

def animate(frame):
    q = []
    for index in range(len(s_q)):
        t = frame / total
        q += [s_q[index] + t * (e_q[index] - s_q[index])]
    chain = irb_chain(q, irb_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)


animate(0)
fps = 25
irb_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(irb_ani.to_jshtml())

Опишите разницу между линейным движнием и переброской:
- с точки зрения алгоритма управления
- с точки зрения исполняемого движения
- с точки зрения изменения обобщенных координат (постройте графики)

Заранее дан способ построения графиков для переброски

In [ ]:
total = 20
step = 0.01
t = np.arange(0, total, step)

fig = plt.figure()
ax = fig.add_subplot()
s_q = irb_ik_lim(s, irb_l, irb_i)
e_q = irb_ik_lim(e, irb_l, irb_i)
q = []
for index in range(6):
    q += [s_q[index] + t / total * (e_q[index] - s_q[index])]
ax.plot(t, q[0], label="$q_0$")
ax.plot(t, q[1], label="$q_1$")
ax.plot(t, q[2], label="$q_2$")
ax.plot(t, q[3], label="$q_3$")
ax.plot(t, q[4], label="$q_4$")
ax.plot(t, q[5], label="$q_5$")
fig.legend()
fig.show()

Напишите свою реализацию для линейного движения (ниже есть реализация для оценки скоростей, можете воспользоваться ей, адаптировав ее).

Напишем функцию для объединения двух линейных движений:

In [ ]:
def lin_lin(start, inter, end, t, total):
    progress = t / total
    if progress < 0.5:
        return Transform.lerp(
            start,
            inter,
            progress * 2
        )
    else:
        return Transform.lerp(
            inter,
            end,
            (progress - 0.5) * 2
        )

Добавим промежуточную точку:

In [ ]:
i = Transform(
    Vector(400, 100, 900),
    Quaternion.from_angle_axis(np.pi / 4, Vector(0, 1, 0))
)

Рассмотрим два линейных движения в цепочке:

In [ ]:
(x, y, z) = graphics.chain_to_points(
    irb_chain([0, 0, 0, 0, 0, 0], irb_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s, 100)
graphics.axis(ax, i, 100)
graphics.axis(ax, e, 100)

total = 100

def animate(frame):
    trs = lin_lin(s, i, e, frame, total)
    q = irb_ik_lim(
        trs,
        irb_l,
        irb_i
    )
    if q != None:
        chain = irb_chain(q, irb_l)
        (x, y, z) = graphics.chain_to_points(chain)
        lines.set_data_3d(x, y, z)


animate(0)
fps = 25
irb_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(irb_ani.to_jshtml())

Проанализируем скорости обобщенных координат:

In [ ]:
v_lin_lin = np.vectorize(lin_lin, excluded={0, 1, 2, 4})
v_irb_ik = np.vectorize(irb_ik_lim, excluded={1, 2})
total = 20
step = 0.01
t = np.arange(0, total, step)

fig = plt.figure()
ax = fig.add_subplot()
w = np.diff(v_irb_ik(
    v_lin_lin(s, i, e, t, total),
    irb_l,
    irb_i
)) / step;
ax.plot(t[:-1], w[0], label="$\omega_0$")
ax.plot(t[:-1], w[1], label="$\omega_1$")
ax.plot(t[:-1], w[2], label="$\omega_2$")
ax.plot(t[:-1], w[3], label="$\omega_3$")
ax.plot(t[:-1], w[4], label="$\omega_4$")
ax.plot(t[:-1], w[5], label="$\omega_5$")
fig.legend()
fig.show()

Как меняется скорость приводов?

Напишем функцию для объединения двух линейных движений со сглаживанем:

In [ ]:
def bezier_transform(a, b, c, t):
    return Transform.lerp(
        Transform.lerp(a, b, t),
        Transform.lerp(b, c, t),
        t
    )

def lin_lin_smooth(start, inter, end, t, total, blend=0.1):
    progress = t / total
    if np.abs(progress - 0.5) < blend:
        progress = (progress - 0.5 + blend) / 2 / blend
        a = lin(start, inter, 1.0 - 2 * blend, 1)
        b = inter
        c = lin(inter, end, 2 * blend, 1)
        return bezier_transform(
            a,
            b,
            c,
            progress
        )
    else:
        return lin_lin(start, inter, end, t, total)

In [ ]:
blending = 0.1

In [ ]:
(x, y, z) = graphics.chain_to_points(
    irb_chain([0, 0, 0, 0, 0, 0], irb_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s, 100)
graphics.axis(ax, i, 100)
graphics.axis(ax, e, 100)

total = 100

def animate(frame):
    trs = lin_lin_smooth(s, i, e, frame, total, 0.1)
    q = irb_ik_lim(
        trs,
        irb_l,
        irb_i
    )
    if q != None:
        chain = irb_chain(q, irb_l)
        (x, y, z) = graphics.chain_to_points(chain)
        lines.set_data_3d(x, y, z)


animate(0)
fps = 25
irb_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(irb_ani.to_jshtml())

Снова проанализируем скорости обобщенных координат:

In [ ]:
v_lin_lin = np.vectorize(lin_lin_smooth, excluded={0, 1, 2, 4, 5})
v_irb_ik = np.vectorize(irb_ik_lim, excluded={1, 2})
total = 20
step = 0.01
t = np.arange(0, total, step)

fig = plt.figure()
ax = fig.add_subplot()
w = np.diff(v_irb_ik(
    v_lin_lin(s, i, e, t, total, blending),
    irb_l,
    irb_i
)) / step;
ax.plot(t[:-1], w[0], label="$\omega_0$")
ax.plot(t[:-1], w[1], label="$\omega_1$")
ax.plot(t[:-1], w[2], label="$\omega_2$")
ax.plot(t[:-1], w[3], label="$\omega_3$")
ax.plot(t[:-1], w[4], label="$\omega_4$")
ax.plot(t[:-1], w[5], label="$\omega_5$")
fig.legend()
fig.show()

Как меняется скорость приводов?

## Самостоятельное задание:
Проведите исследование для своих точек траектории.

Проанализируйте влияние параметра `blend` на скорость обобщенных координат.

Оцените ускорения обобщенных координат (через дифференцирование второго порядка `diff(..., 2`).